In [1]:
import h5py
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from env_vars import *

In [2]:
# from google.colab import drive

# drive.mount("/content/gdrive")

In [3]:
# Used to get the name of the dataset, and by extension, the label of the action being performed
def getdatasetname(file_name_with_dir):
    filename_without_dir = file_name_with_dir.split("/")[-1]


    temp = filename_without_dir.split("_")[:-1]

    datasetname = "_".join(temp)

    return datasetname

In [4]:
# Used to get the labels of data using filenames
def get_label(filename):
    if "rest" in filename:

        label = 0

    elif "math" in filename:
        label = 1

    elif "memory" in filename:
        label = 2
    elif "motor" in filename:

        label = 3

    return label

In [5]:
def get_all_matrices(dir_path):
    dataset = []
    labels = []
    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = dir_path + filename
            with h5py.File(filename_path, "r") as f:
                dataset_name = getdatasetname(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]
                dataset.append(matrix)
                labels.append(label)

    return dataset, labels

MinMax Scaling for sensor data


In [6]:
def scale(matrix):
    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(matrix)

    scaled_data = scaler.transform(matrix)

    return scaled_data

Downsampling of data, refer to file called "Notebook.ipynb" to see how it works


In [7]:
def downsample(dataset, frequency):
    downsampled_dataset = []

    for i in range(0, dataset.shape[1], 2034):
        second = dataset[:, i : i + 2034]
        subsample = []


        for j in range(0, 2034, int(2034 / frequency)):
            if j < second.shape[1]:
                measurement = second[:, j]
                subsample.append(measurement)


        downsampled_dataset.extend(subsample)


    return np.array(downsampled_dataset).T

Model setup


In [8]:
# TensorFlow and tf.keras
import tensorflow as tf

print(tf.__version__)

2.15.0


This is where the archtiecture is specified. For now it is just a Recurrent Neural Network (RNN) with two layers with 128 parameters each i.e., not very sophisticated.


In [9]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            128, activation="relu"
        ),  # The layer has 128 dense neurons using relu for activation
        tf.keras.layers.Dense(
            128, activation="relu"
        ),  # The layer has 128 dense neurons using relu for activation
        tf.keras.layers.Dense(
            4
        ),  # The final layer has 4 neurons, each representing a class for the action being done
    ]
)

In [10]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [11]:
def load_data(dir_path):
    data = []
    labels = []

    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = os.path.join(dir_path, filename)
            with h5py.File(filename_path, "r") as f:
                dataset_name = getdatasetname(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]

                train_meg = downsample(matrix, 113)
                train_meg = scale(train_meg)
                flattened_meg = np.array(
                    train_meg.flatten()
                )  # The data is flattened to change the meg data from shape 248 x frequency to a 1D array (result: a lot of inupt params)

                data.append(flattened_meg)
                labels.append(label)

    return np.array(data), np.array(labels)

All training data is loaded into memory and fitted to the model along with the training lables


In [12]:
def train_model(dir_path, model):
    train_data, train_labels = load_data(dir_path)

    model.fit(train_data, train_labels, epochs=10)

In [13]:
def test_model(dir_path, model):
    test_data, test_labels = load_data(dir_path)

    test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)

    return test_loss, test_acc

In [14]:
train_model(intra_train_path, model)

NameError: name 'intra_train_path' is not defined

In [40]:
test_loss, test_acc = test_model(intra_test_path, model)

print("\nTest accuracy:", test_acc)

1/1 - 0s - loss: 201.1091 - accuracy: 0.2500 - 214ms/epoch - 214ms/step

Test accuracy: 0.25
